In [1]:
import pandas as pd
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
from scipy.interpolate import InterpolatedUnivariateSpline as Spline
import matplotlib as mpl
import seaborn as sns

In [2]:
sipp_2014_2021 = pd.read_csv(r"C:\Users\shell\OneDrive\OrganizeMe\Documents\GitHub\studentdebt\SIPP data\SIPP_2014_2021.txt", index_col=0)
datalab = pd.read_csv(r"D:\NCES DataLab Data\DataLab_undergrad_completed.csv")

C:\Users\shell\AppData\Local\Temp\ipykernel_28432\2729473935.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  sipp_2014_2021 = pd.read_csv(r"C:\Users\shell\OneDrive\OrganizeMe\Documents\GitHub\studentdebt\SIPP data\SIPP_2014_2021.txt", index_col=0)


Functions intended for use with the SIPP data:

In [3]:
def median(dataframe, independent, dependent, point):
    '''
    Finds the median value of the dependent variable at the point given for the independent variable
    '''
    df = dataframe
    df = df[df[independent] == point]
    if df[dependent].median() > 0:
        return df[dependent].median()
    else:
        return 0

In [4]:
def median_sort_by(dataframe, independent, dependent):
    '''
    Sorts a dataframe such that it compares the indpendent variable to the median of the dependent variable
    '''
    df = dataframe
    df[independent] = df[independent].fillna(-1)
    df[independent] = df[independent].astype('int')
    df = df[(df['MONTHCODE'] == 1) & (df['EEDUC'] == 42) & (df[dependent] > 0) & (df[independent] > 0)]
    
    new_df = pd.DataFrame()
    new_df[independent] = range(df[independent].min(), (df[independent].max()+1))
    median_list = []
    for point in new_df[independent]:
        median_list.append(median(dataframe, independent, dependent, point))

    new_df['median ' + dependent] = median_list
    
    no_data_independent = []
    for x in new_df[independent]:
        if len(df[(df[dependent] == x)].index) < 10:
            no_data_independent.append(x)
    # new_df.drop(index=no_data_independent, axis='index', inplace=True)
    return new_df

In [5]:
def median_bar_graph(dataframe, independent, dependent, title, xlab, ylab):
    df = median_sort_by(dataframe, independent, dependent)
    plt.bar(df[independent], df['median ' + dependent])
    plt.title(title)
    plt.xlabel(xlab)
    plt.ylabel(ylab)
    plt.show()

Functions intended for use with the NCES's NPSAS 2015-2016 data:

In [6]:
def datalab_round(dataframe):
    '''
    Rounds all values of the data to remove floating points
    '''
    df = dataframe
    df = df.fillna(-1)
    for x in range(3, len(df.index)):
        for y in range(1, len(df.columns)):
            df.iloc[x, y] = int(round(float(df.iloc[x, y]), 2))
    return df

In [7]:
def datalab_bar_graph(column, row_range, title, xlab, ylab):
    '''
    column must be a column from the dataframe, row_range must be a range() function
    '''
    
    plt.bar(datalab['Variable'].iloc[row_range], 
    datalab[datalab.iloc[:, range(1, 6)] != -1].iloc[row_range, datalab.columns.get_loc(column)])
    # discards -1 values from rounding process, replacing them with NAN values that are automaatically ignored by Pandas 
    
    plt.title(title)
    plt.xlabel(xlab)
    plt.ylabel(ylab)
    plt.show